In [ ]:
# lambda.py

# Serialize Image Data
import json
import boto3
import base64

s3 = boto3.client('s3')

def serialize_image_data(event, context):
    key = event["s3_key"]
    bucket = event["s3_bucket"]
    s3.download_file(bucket, key, "/tmp/image.png")
    with open("/tmp/image.png", "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
    return {
        'statusCode': 200,
        'body': {
            "image_data": image_data,
            "s3_bucket": bucket,
            "s3_key": key,
            "inferences": []
        }
    }

# Image Classification
import json
import boto3
import base64
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer

ENDPOINT = 'your-endpoint-name'

def classify_image(event, context):
    image = base64.b64decode(event["body"]["image_data"])
    predictor = Predictor(endpoint_name=ENDPOINT)
    predictor.serializer = IdentitySerializer("image/png")
    inferences = predictor.predict(image).decode('utf-8')
    event["body"]["inferences"] = json.loads(inferences)
    return {
        'statusCode': 200,
        'body': event["body"]
    }

# Filter Inferences
import json

THRESHOLD = 0.93

def filter_inferences(event, context):
    inferences = event["body"]["inferences"]
    meets_threshold = any([float(confidence) > THRESHOLD for confidence in inferences])
    if meets_threshold:
        pass
    else:
        raise Exception("THRESHOLD_CONFIDENCE_NOT_MET")
    return {
        'statusCode': 200,
        'body': event["body"]
    }
